# 4YFN WEBPAGE SCRAPER
A scraper for the exhibitors webpage: https://www.mwcbarcelona.com/exhibition/2019-exhibitors/

In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time 

In [2]:
class Exhibitor:
    def __init__(self, link):
        self.link = link
        self.name = link[31:-1]
        self.location = None
        self.description = None
        self.email = None
        self.phone = None
        self.website = None
    
    def __str__(self):
        return f'''
        Name: {self.name} 
        Location: {self.location}
        Description: {self.description[:50]}...
        Email: {self.email}
        Website: {self.website}
        '''
    def __repr__(self):
        return f'''
        Name: {self.name} 
        Location: {self.location}
        Description: {'No description' if not self.description else self.description[:50]}...
        Email: {self.email}
        Website: {self.website}
        Number: {self.phone}
        Link: {self.link}
        '''
    def to_list(self):
        return [self.name, self.link, self.location, self.phone, self.email, self.website, self.description]
        

In [3]:
def get_links(tipus):
    """This function acceses the main exhibitors webpage and gets a list with all the individual links
    
    Returns: list with individual links"""
    
    #list to store the links
    name_list = list()
    
    url = 'https://www.4yfn.com/barcelona/2019-exhibitors/page/2/'
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}

    for i in range(77):
        #request
        if tipus=='mwc':
            r = requests.get(f'https://www.mwcbarcelona.com/exhibition/2019-exhibitors/page/{i}', headers=headers)
        elif tipus=='4yfn':
            r = requests.get(f'https://www.4yfn.com/barcelona/2019-exhibitors/page/{i}', headers=headers)
        content = r.text
        
        #regex
        name_pattern = re.compile(r'https://www.4yfn.com/exhibitor/[\w-]+/')
        names = name_pattern.findall(content)

        name_list.extend(names)
    
    return name_list

In [24]:
def get_info(link):
    """Gets all the information in the page
    
    Params: link = It is the page you want to scrape
    Returns: Exhibitor object = An object with all the information"""
    
    #create object
    ex = Exhibitor(link)
    
    #request and prep
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    headers = {'User-Agent': user_agent}
    r = requests.get(link, headers=headers)
    content = r.content
    text = r.text
    soup = BeautifulSoup(content)
    
    #find location
    name_pattern = re.compile(r'Hall 4YFN [\w\s]+')
    location = name_pattern.findall(text)
    if location:
        ex.location = location[0]
    
    #find description
    div_api_description = soup.find("div", {'class':'api-description'})
    if div_api_description:
        p_description = list(div_api_description.children)[1]
        description = list(p_description.children)
        if description:
            ex.description = description[0]
    
    #find phone
    p_contact_details = soup.find("p", string='Contact Details')
    if p_contact_details:
        parent_div = p_contact_details.parent.parent
        search = re.findall('[-+\d() ]+',parent_div.text)
        if search:
            ex.phone = search[1]
    
    #find email
    email_pattern = re.compile(r'mailto:[-a-zA-Z0-9_.+]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+')
    email = email_pattern.findall(text)
    if email:
        ex.email = email[0][7:]
    
    #find website
    name_pattern = re.compile(r'class="web-site-link"\shref="http://www.[a-zA-Z0-9-.]+.[a-zA-Z0-9-.]+"')
    website = name_pattern.findall(str(soup))
    if website:
        ex.website = website[0][28:-1]
    
    return ex
    
    

In [5]:
def fetch_all(list_of_links, number=None):
    """Scrapes all the pages from list of links and returns a list with all the exhibitors"""
    info = list()
    i = 0
    if number!=None:
        links = list_of_links[0:number]
    else:
        links = list_of_links
    for link in links:
        print(link)
        i += 1
        if i%10 == 0:
            print(f"Dowloading exhibitor number {i}...")
        info.append(get_info(link))
    return info

In [6]:
def to_df(all_info):
    """Exports the list with all the exhibitors objects to a Dataframe""" 
    list_of_lists = [ex.to_list() for ex in all_info]
    df = pd.DataFrame(list_of_lists, columns = ['Name', 'Link', 'Location', 'Number', 'Email', 'Website', 'Description'])
    return df

In [7]:
links = get_links(tipus="4yfn")

In [8]:
len(links)

825

In [9]:
all_info = fetch_all(links)

https://www.4yfn.com/exhibitor/10xmotion/
https://www.4yfn.com/exhibitor/3dlook/
https://www.4yfn.com/exhibitor/4yfn-awards/
https://www.4yfn.com/exhibitor/4yfn-community-club/
https://www.4yfn.com/exhibitor/abitari/
https://www.4yfn.com/exhibitor/able-human-motion/
https://www.4yfn.com/exhibitor/accelleran/
https://www.4yfn.com/exhibitor/accesslab-it-services/
https://www.4yfn.com/exhibitor/acero-tecnologica-sl/
https://www.4yfn.com/exhibitor/acqustic/
Dowloading exhibitor number 10...
https://www.4yfn.com/exhibitor/acromove-inc/
https://www.4yfn.com/exhibitor/adamo-robot/
https://www.4yfn.com/exhibitor/adan-medical-innovation/
https://www.4yfn.com/exhibitor/adipsys/
https://www.4yfn.com/exhibitor/adsumnet/
https://www.4yfn.com/exhibitor/adzuki/
https://www.4yfn.com/exhibitor/aeinnova/
https://www.4yfn.com/exhibitor/aetolls/
https://www.4yfn.com/exhibitor/africa-water-enterprises/
https://www.4yfn.com/exhibitor/african-solar-designs/
Dowloading exhibitor number 20...
https://www.4yfn.

https://www.4yfn.com/exhibitor/cathay-innovation/
https://www.4yfn.com/exhibitor/catoz/
https://www.4yfn.com/exhibitor/ccei/
https://www.4yfn.com/exhibitor/centro-virtual-de-negocios-s-a-s/
https://www.4yfn.com/exhibitor/challenge-k-startup-2018/
Dowloading exhibitor number 170...
https://www.4yfn.com/exhibitor/checkealos/
https://www.4yfn.com/exhibitor/chiara/
https://www.4yfn.com/exhibitor/chilmyeongbio-co-ltd/
https://www.4yfn.com/exhibitor/china-communications-consultants-sl/
https://www.4yfn.com/exhibitor/chiper/
https://www.4yfn.com/exhibitor/ciclogreen-move-and-win-s-l/
https://www.4yfn.com/exhibitor/city-taps/
https://www.4yfn.com/exhibitor/civic-technologies-inc/
https://www.4yfn.com/exhibitor/ck-materials-lab-co-ltd/
https://www.4yfn.com/exhibitor/clic-for-rent/
Dowloading exhibitor number 180...
https://www.4yfn.com/exhibitor/climbista/
https://www.4yfn.com/exhibitor/clx-communications-ltd/
https://www.4yfn.com/exhibitor/cna/
https://www.4yfn.com/exhibitor/codingbird/
https:

https://www.4yfn.com/exhibitor/fundacion-respol/
https://www.4yfn.com/exhibitor/fundacion-ship2b/
https://www.4yfn.com/exhibitor/fundecyt-pctex/
https://www.4yfn.com/exhibitor/fundeen/
https://www.4yfn.com/exhibitor/furgo/
https://www.4yfn.com/exhibitor/futuresisnes-s-l/
https://www.4yfn.com/exhibitor/g1-partners-inc/
https://www.4yfn.com/exhibitor/gala-urban-food-sl/
Dowloading exhibitor number 330...
https://www.4yfn.com/exhibitor/garage-51/
https://www.4yfn.com/exhibitor/gemmy-company-inc/
https://www.4yfn.com/exhibitor/generalitat-de-catalunya/
https://www.4yfn.com/exhibitor/george-and-shaun-llc/
https://www.4yfn.com/exhibitor/ges-emer/
https://www.4yfn.com/exhibitor/gham-power-nepal-pvt-ltd/
https://www.4yfn.com/exhibitor/giantfox/
https://www.4yfn.com/exhibitor/giftedmom/
https://www.4yfn.com/exhibitor/global-citizens-debate-on-the-future-of-internet/
https://www.4yfn.com/exhibitor/globe-testing/
Dowloading exhibitor number 340...
https://www.4yfn.com/exhibitor/glue-digital/
http

https://www.4yfn.com/exhibitor/merlot-laboratories/
https://www.4yfn.com/exhibitor/messagenius/
https://www.4yfn.com/exhibitor/metropole-ruhr/
https://www.4yfn.com/exhibitor/microriobiotic-sl/
https://www.4yfn.com/exhibitor/microsystems-inc/
https://www.4yfn.com/exhibitor/minalogic/
https://www.4yfn.com/exhibitor/mindtech-apps/
https://www.4yfn.com/exhibitor/minority-startups-ltd/
https://www.4yfn.com/exhibitor/minushu/
https://www.4yfn.com/exhibitor/mira-robotics/
Dowloading exhibitor number 490...
https://www.4yfn.com/exhibitor/miraxess/
https://www.4yfn.com/exhibitor/miro-in-cube-programme-miro/
https://www.4yfn.com/exhibitor/mitsuya-co-ltd/
https://www.4yfn.com/exhibitor/moasure/
https://www.4yfn.com/exhibitor/mobietrain/
https://www.4yfn.com/exhibitor/mobilous-inc/
https://www.4yfn.com/exhibitor/mobisol/
https://www.4yfn.com/exhibitor/modularity-grid/
https://www.4yfn.com/exhibitor/moin-inc/
https://www.4yfn.com/exhibitor/monkingme-s-l/
Dowloading exhibitor number 500...
https://w

https://www.4yfn.com/exhibitor/senorics/
https://www.4yfn.com/exhibitor/sensorial-processing-technology-bcn/
https://www.4yfn.com/exhibitor/sheetgo/
Dowloading exhibitor number 650...
https://www.4yfn.com/exhibitor/shieldiot/
https://www.4yfn.com/exhibitor/shopitag/
https://www.4yfn.com/exhibitor/shotl/
https://www.4yfn.com/exhibitor/siba/
https://www.4yfn.com/exhibitor/siemens-mobility-slu/
https://www.4yfn.com/exhibitor/sigma-sar-institute-llc/
https://www.4yfn.com/exhibitor/simcar/
https://www.4yfn.com/exhibitor/simgas/
https://www.4yfn.com/exhibitor/singerhood/
https://www.4yfn.com/exhibitor/singularbrain-consulting-s-l/
Dowloading exhibitor number 660...
https://www.4yfn.com/exhibitor/situm-indoor-positioning/
https://www.4yfn.com/exhibitor/sk-telecom-co-ltd/
https://www.4yfn.com/exhibitor/skyeye-pacific/
https://www.4yfn.com/exhibitor/slast-chuchy/
https://www.4yfn.com/exhibitor/smarf-co-ltd/
https://www.4yfn.com/exhibitor/smart-biosystem/
https://www.4yfn.com/exhibitor/smart-iot

https://www.4yfn.com/exhibitor/wivi-by-e-health-technical-solutions/
Dowloading exhibitor number 810...
https://www.4yfn.com/exhibitor/wonderbits/
https://www.4yfn.com/exhibitor/workdeck/
https://www.4yfn.com/exhibitor/xapix/
https://www.4yfn.com/exhibitor/xcale3d/
https://www.4yfn.com/exhibitor/xente/
https://www.4yfn.com/exhibitor/xpik-universal-language/
https://www.4yfn.com/exhibitor/yo-tambien-leo/
https://www.4yfn.com/exhibitor/youcheck/
https://www.4yfn.com/exhibitor/youthstarting/
https://www.4yfn.com/exhibitor/ysjeneral-co-ltd/
Dowloading exhibitor number 820...
https://www.4yfn.com/exhibitor/zain-innovation-campus-zinc/
https://www.4yfn.com/exhibitor/zero-2-infinity/
https://www.4yfn.com/exhibitor/zigzag-global/
https://www.4yfn.com/exhibitor/zuratrust-all-your-emails-verified/
https://www.4yfn.com/exhibitor/zymvol-biomodeling-s-l/


In [32]:
df = to_df(all_info)

In [33]:
df.to_csv("./exhibitors_4yfn.csv")

In [14]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options

In [154]:
df = pd.read_csv("./4yfn.csv")

In [155]:
df = df.where((pd.notnull(df)), None)

In [10]:
def from_df_to_list(df):
    all_info = list()
    for i in range(len(df)):
        row = df.iloc[i]
        values = row.values
        ex = Exhibitor(values[2])
        ex.location = values[3]
        ex.description = values[7]
        ex.phone = values[4]
        all_info.append(ex)
    
    return all_info
    

In [157]:
all_info = from_df_to_list(df)

In [11]:
def get_other_info_headless(browser, ex):
    browser.get(ex.link)
    print(ex.link)
    try:
        elem = browser.find_element_by_css_selector('div.social-exhibitors:nth-child(1)')
    except:
        print(f"Couldn't find info for {ex.name}")
        return
    email_elem = elem.find_element_by_class_name('email-link')
    link_elem = elem.find_element_by_class_name('web-site-link')
    email = email_elem.get_attribute('href')
    if len(email)>2:
        ex.email = email[7:]
    web = link_elem.get_attribute('href')
    if len(web)>2:
        ex.website = web

In [15]:
def complete_all_info(all_info, start):
    try:
        print(f'New instance starting at {start}')
        opts = Options()
        opts.set_headless()
        assert opts.headless  # Operating in headless mode
        browser = Firefox(options=opts)
        time.sleep(3)
        i=0
        for ex in all_info:
            get_other_info_headless(browser, ex)
            i += 1
            if i%10==0: print(f'Downloading {i}th exhibitor...')
            if i%100==0: 
                browser.close()
                complete_all_info(all_info[start+i:],start+i)
        print("Finished!")
    except:
        browser.close
        print('Error')
         #complete_all_info(all_info[i+1:])
        
    

In [31]:
complete_all_info(all_info[200:],0)

New instance starting at 0


/home/jc/miniconda3/envs/scraper/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use setter for headless property instead of set_headless
  """


https://www.4yfn.com/exhibitor/cover-digital-sl/
Couldn't find info for cover-digital-sl
https://www.4yfn.com/exhibitor/coxspace/
https://www.4yfn.com/exhibitor/creatsens-health/
https://www.4yfn.com/exhibitor/creperia-school-bus/
Couldn't find info for creperia-school-bus
https://www.4yfn.com/exhibitor/cryptpad-xwiki-sas/
Couldn't find info for cryptpad-xwiki-sas
https://www.4yfn.com/exhibitor/cuatrecasas-acelera/
https://www.4yfn.com/exhibitor/cuideo/
https://www.4yfn.com/exhibitor/cvicloud-technology/
https://www.4yfn.com/exhibitor/cyberarmor/
https://www.4yfn.com/exhibitor/cyberus-labs-sp-z-o-o/
https://www.4yfn.com/exhibitor/d-light/
Couldn't find info for d-light
https://www.4yfn.com/exhibitor/dada-career/
https://www.4yfn.com/exhibitor/dae-engineering-and-product-development/
https://www.4yfn.com/exhibitor/daegu-digital-industry-promotion-agency-dip/
Couldn't find info for daegu-digital-industry-promotion-agency-dip
https://www.4yfn.com/exhibitor/daejeon-center-for-creative-econ

/home/jc/miniconda3/envs/scraper/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use setter for headless property instead of set_headless
  """


https://www.4yfn.com/exhibitor/field-buzz/
https://www.4yfn.com/exhibitor/film-valley/
https://www.4yfn.com/exhibitor/filmclub/
https://www.4yfn.com/exhibitor/fin-insight/
https://www.4yfn.com/exhibitor/finboot/
https://www.4yfn.com/exhibitor/findalia-com/
https://www.4yfn.com/exhibitor/flexvpc/
https://www.4yfn.com/exhibitor/flugilo/
https://www.4yfn.com/exhibitor/followhealth/
https://www.4yfn.com/exhibitor/fonepay-esewa/
Couldn't find info for fonepay-esewa
https://www.4yfn.com/exhibitor/foods-for-tomorrow/
https://www.4yfn.com/exhibitor/forciot-ltd/
https://www.4yfn.com/exhibitor/foru/
https://www.4yfn.com/exhibitor/frizbit-inc/
https://www.4yfn.com/exhibitor/fuelium/
https://www.4yfn.com/exhibitor/fuelsave/
https://www.4yfn.com/exhibitor/fundacio-bosch-i-gimpera/
Couldn't find info for fundacio-bosch-i-gimpera
https://www.4yfn.com/exhibitor/fundacio-esade/
https://www.4yfn.com/exhibitor/fundacio-tic-salut-social/
https://www.4yfn.com/exhibitor/fundacio-universitaria-balmes/
Couldn

/home/jc/miniconda3/envs/scraper/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use setter for headless property instead of set_headless
  """


https://www.4yfn.com/exhibitor/more-stamina-sl/
Couldn't find info for more-stamina-sl
https://www.4yfn.com/exhibitor/mosaic-factor/
https://www.4yfn.com/exhibitor/most-inc/
https://www.4yfn.com/exhibitor/motion-s-s-a/
https://www.4yfn.com/exhibitor/motorica/
https://www.4yfn.com/exhibitor/mouters/
https://www.4yfn.com/exhibitor/movilest/
https://www.4yfn.com/exhibitor/mowoot/
https://www.4yfn.com/exhibitor/mschools-sdg-challenge/
https://www.4yfn.com/exhibitor/musanga-logistics/
https://www.4yfn.com/exhibitor/musiclist/
https://www.4yfn.com/exhibitor/must-travel-tech/
https://www.4yfn.com/exhibitor/muutech-monitoring-solutions/
https://www.4yfn.com/exhibitor/mydata-global/
Couldn't find info for mydata-global
https://www.4yfn.com/exhibitor/myhealth-watcher/
https://www.4yfn.com/exhibitor/mymanu-play/
https://www.4yfn.com/exhibitor/n-e-t-c-r-y-o/
https://www.4yfn.com/exhibitor/n26/
https://www.4yfn.com/exhibitor/nacar-design/
https://www.4yfn.com/exhibitor/nanolock-security/
https://ww

/home/jc/miniconda3/envs/scraper/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use setter for headless property instead of set_headless
  """


https://www.4yfn.com/exhibitor/wedding/
Couldn't find info for wedding
https://www.4yfn.com/exhibitor/wefish-fishing-diary-and-forecast/
https://www.4yfn.com/exhibitor/wegoto/
https://www.4yfn.com/exhibitor/welcome20/
https://www.4yfn.com/exhibitor/white-label-loyalty/
https://www.4yfn.com/exhibitor/whoopee-app-s-l/
Couldn't find info for whoopee-app-s-l
https://www.4yfn.com/exhibitor/wirelesscities-networks-s-l/
https://www.4yfn.com/exhibitor/witekio/
https://www.4yfn.com/exhibitor/witrac-soluciones-inteligentes-s-l/
https://www.4yfn.com/exhibitor/wivi-by-e-health-technical-solutions/
https://www.4yfn.com/exhibitor/wonderbits/
https://www.4yfn.com/exhibitor/workdeck/
https://www.4yfn.com/exhibitor/xapix/
https://www.4yfn.com/exhibitor/xcale3d/
https://www.4yfn.com/exhibitor/xente/
https://www.4yfn.com/exhibitor/xpik-universal-language/
https://www.4yfn.com/exhibitor/yo-tambien-leo/
https://www.4yfn.com/exhibitor/youcheck/
https://www.4yfn.com/exhibitor/youthstarting/
https://www.4yfn.

In [28]:
all_info[600:]

[
         Name: prismab 
         Location: Hall 4YFN Montjuic Hall M8 Stand J3
         Description: PRISMAB is an AgroTech company. We provide the mos...
         Email: antonio@prismab.com
         Website: http://www.prismab.com/
         Number: +34 619144343
         Link: https://www.4yfn.com/exhibitor/prismab/
         , 
         Name: pro4people-sp-z-o-o 
         Location: Hall 4YFN Montjuic Hall M8 Stand D2
         Description: No description...
         Email: None
         Website: None
         Number: None
         Link: https://www.4yfn.com/exhibitor/pro4people-sp-z-o-o/
         , 
         Name: procolombia 
         Location: Hall 4YFN Montjuic Hall M8 Stand E3
         Description: PROCOLOMBIA is the entity that promotes internatio...
         Email: duscategui@procolombia.co
         Website: http://www.procolombia.co/
         Number: +34 915776708
         Link: https://www.4yfn.com/exhibitor/procolombia/
         , 
         Name: propcrowd 
         Location

## Send EMAIL

In [184]:
import smtplib
from email.mime.text import MIMEText
import cred

def send_emails(all_info, user, password):
    i = 0
    #Acces credentials
    gmail_user = user 
    gmail_password = password


    #Server set up
    server = smtplib.SMTP('smtp.gmail.com:587')
    server.starttls()
    server.ehlo()
    server.login(gmail_user, gmail_password)
    
    #send_email(server, 'jaume', 'jaumecolomhernandez@gmail.com')
    
    #Send email
    for ex in all_info:
        if ex.email!=None:
            time.sleep(1)
            send_email(server, ex.name, ex.email)
            print(f"Email sent to: {ex.name}")
            i += 1
            if i%10 == 0: print(f"{i}th email sent")
    print(f"Finished -- {i} emails sent")

    server.quit()


In [185]:
def send_email(server, name, email):
    email_text = f"""
Dear {name.capitalize()},

My name is Jaume Colom, I am studying at the technical university of Catalonia and part of the ForumTIC organizing team, a fair in which the best tech companies get in touch with computer science and telecommunications students.

I am writing you because next week starts the 4YFN and I would really like to assist. Especially because I think that it is a great place to meet with companies and learn. That is why I would like to know whether it is possible to get a ticket for the 4YFN. Thanks for your time and your attention.

Best regards,

Jaume
"""
    msg = MIMEText(email_text)
    msg['Subject'] = '4YFN Contact'
    msg['From'] = 'jaume.colom@forumtic.upc.edu'
    msg['To'] = email
    
    server.sendmail('jaume.colom@forumtic.upc.edu', email, msg.as_string())

In [182]:
all_info


[
         Name: 10xmotion 
         Location: Hall 4YFN Montjuic Hall M8 Stand P4
         Description: 10XMotion is building a company to design and manu...
         Email: thomas.lloyd@10xbeta.com
         Website: http://10xmotion.com/
         Number: (076) 630-6047
         Link: https://www.4yfn.com/exhibitor/10xmotion/
         , 
         Name: 3dlook 
         Location: Hall 4YFN Montjuic Hall M8 Stand 20
         Description: 3DLOOK is a pioneer of mobile body scanning techno...
         Email: None
         Website: https://3dlook.me/
         Number: None
         Link: https://www.4yfn.com/exhibitor/3dlook/
         , 
         Name: 4yfn-awards 
         Location: Hall 4YFN Montjuic Hall M8 Stand 20
         Description: No description...
         Email: None
         Website: None
         Number: None
         Link: https://www.4yfn.com/exhibitor/4yfn-awards/
         , 
         Name: 4yfn-community-club 
         Location: Hall 4YFN Montjuic Hall M8 Stand N1
        

In [186]:
send_emails(all_info, cred.gmail_user, cred.gmail_password)

Email sent to: 10xmotion
Email sent to: able-human-motion
Email sent to: accesslab-it-services
Email sent to: acero-tecnologica-sl
Email sent to: acqustic
Email sent to: adamo-robot
Email sent to: adan-medical-innovation
Email sent to: adipsys
Email sent to: adsumnet
Email sent to: adzuki
10th email sent
Email sent to: aeinnova
Email sent to: aetolls
Email sent to: afterbanks
Email sent to: agora-io
Email sent to: ahqlab-inc
Email sent to: airbus
Email sent to: airhop-communications
Email sent to: airning
Email sent to: allblanc
Email sent to: allergeneat
20th email sent
Email sent to: altair
Email sent to: alteria-automation
Email sent to: altheria-solutions
Email sent to: amant-antennas
Email sent to: amarisoft
Email sent to: ambeent
Email sent to: ancora
Email sent to: antidote
Email sent to: antiloop-studio
Email sent to: apercu
30th email sent
Email sent to: applied-blockchain
Email sent to: aquabit-spirals
Email sent to: armet-gmbh
Email sent to: argeo
Email sent to: art-map
Emai

Email sent to: modularity-grid
260th email sent
Email sent to: moin-inc
Email sent to: monkingme-s-l
Email sent to: most-inc
Email sent to: motion-s-s-a
Email sent to: mouters
Email sent to: mowoot
Email sent to: mschools-sdg-challenge
Email sent to: musanga-logistics
Email sent to: musiclist
Email sent to: must-travel-tech
270th email sent
Email sent to: muutech-monitoring-solutions
Email sent to: myhealth-watcher
Email sent to: nacar-design
Email sent to: nanolock-security
Email sent to: napptilus-tech-labs
Email sent to: navin
Email sent to: neo
Email sent to: nestech-corporation
Email sent to: neubility-inc
Email sent to: neurafy
280th email sent
Email sent to: neurodigital-technologies-sl
Email sent to: neurofinance-technologies-s-l
Email sent to: new-day
Email sent to: newton1665-co-ltd
Email sent to: nirvine-kpi
Email sent to: nomorepass
Email sent to: novameat
Email sent to: nuclio-venture-builder
Email sent to: nurturewatch
Email sent to: nutrasign
290th email sent
Email sent 

SMTPDataError: (550, b'5.4.5 Daily user sending quota exceeded. c186sm18838858wmf.34 - gsmtp')

In [ ]:
#TODO : Refactor for 4YFN website